After analyzing the image game we found :
    - image 210 x 160 x 3
    - ball 4 x 2
    - pad 4 * 16

We have decided to consider the game like a pong prototype, our goal is to send back the ball.
As we decided to use Q-learning. Our training are made from a discrete environment.
We first have to reduce our environment :
Only keep bottom from the paddle and before the first breakable block (see image -> ....)

In [1]:
import imageio
import numpy as np
import gym
from gym.utils.play import play


def callback(obs_t, obs_tp1, action, rew, done, info):
    # image  50 x 72 x 1 color.
    imageio.imwrite('outfile1.png', obs_t[94:194:2, 8:152:2, 1])
    if action != 0:
        print(info)
        print(action)


env = gym.make('Breakout-v0')
play(env, zoom=3, fps=12, callback=callback)
env.reset()
env.close()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


/usr/local/lib/python3.9/site-packages/gym/utils/play.py:11: UserWarning: WARN: failed to set matplotlib backend, plotting will not work: No module named '_tkinter'
  logger.warn("failed to set matplotlib backend, plotting will not work: %s" % str(e))
A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]
2022-03-20 21:00:29.767 Python[28693:1215164] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/kl/c5lh2mc52hbbmbk_l0df9cvw0000gn/T/org.python.python.savedState


{'lives': 5, 'episode_frame_number': 33, 'frame_number': 33}
1
{'lives': 5, 'episode_frame_number': 37, 'frame_number': 37}
1
{'lives': 5, 'episode_frame_number': 123, 'frame_number': 123}
1
{'lives': 4, 'episode_frame_number': 127, 'frame_number': 127}
1
{'lives': 4, 'episode_frame_number': 221, 'frame_number': 221}
1
{'lives': 4, 'episode_frame_number': 223, 'frame_number': 223}
1
{'lives': 3, 'episode_frame_number': 242, 'frame_number': 242}
1
{'lives': 3, 'episode_frame_number': 245, 'frame_number': 245}
1
{'lives': 3, 'episode_frame_number': 261, 'frame_number': 261}
1
{'lives': 3, 'episode_frame_number': 270, 'frame_number': 270}
1
{'lives': 3, 'episode_frame_number': 277, 'frame_number': 277}
1
{'lives': 3, 'episode_frame_number': 281, 'frame_number': 281}
1
{'lives': 3, 'episode_frame_number': 285, 'frame_number': 285}
1
{'lives': 3, 'episode_frame_number': 289, 'frame_number': 289}
1
{'lives': 3, 'episode_frame_number': 291, 'frame_number': 291}
1
{'lives': 3, 'episode_frame_n

After reduction,we have:
   - image ->  60 x 72 x 1
   - ball 2 x 1
   - pad 2 * 8

Our Q state is composed of :
       B - ball position
       P - pad position
       A - action possible : 3  ( left , stay , right )
       D - direction
N  = BxQxNxD
The final goal will be to try with different composition of B P D . In the aim to find which discretization is more efficiant

Before generalization. Let's make a first example.
We discretize our image :
60 and 72 can be divided both by 12.
For the direction of the ball we take 4 directions, bottom left - bottom right - top left - top right
Our Q state is composed of :
       B - ball position :  5 x 6 position = 30
       P - pad position : 6 position only horizontal
       A - action possible : 3  ( left , stay , right )
       D - direction  : 4
N = 30 x 6 x 3 x 4 = 2160

In [85]:
import numpy as np
import random

XB = 6
YB = 5
P = 6
A = 3
D = 4  #  considering : 0 top-left  1 top right ...
Q = np.zeros((P, XB, YB, D, A), dtype=float)
alpha = 0.8
# x,y,direction
moveBall = {
    0: (1, 1),  # 45 top right
    1: (1, -1),  # -45 bottom right
    2: (-1, -1),  # -135 bottom left
    3: (-1, 1)  # 135 top-left
}


def updateBall(x, y, d, pad):
    #
    xD, yD = moveBall[d]
    nd = d
    nX, nY = x + xD, y + yD

    if pad == nX and nY == 0 or YB - 1 == nY:
        nd = (d + 2) % 4

    # if
    elif (XB - 1 == nX and d == 0) or (nX == 0 and d == 3):
        nd = d + (-3 if XB - 1 == nX and d == 0 else +3)


    elif (XB - 1 == nX and d == 1) or (nX == 0 and d == 2):
        nd = d + (1 if XB - 1 == nX and d == 1 else -1)

    return nX, nY, nd


def rewards(xb, yb, p):
    if p == xb and yb == 0:
        return 3
    return -5 if newYB == 0 and newDB in [1, 2] else 0


...

# iterations

N = 1000
for i in range(N):
    ## Create the pad here random position =
    xPad = 2
    epsilon = 1
    xBall, yBall, dirBall = 2, 2, 3
    endStep = 12000
    while not ((yBall == 0 and dirBall in [1, 2]) or endStep == 0):
        # find the index of the max of the line of Q_state
        maxIndex = np.where(Q[xPad][xBall][yBall][dirBall] == max(Q[xPad][xBall][yBall][dirBall]))[0]
        #    np.argmax(Q[xPad][xBall][yBall][dirBall])

        # e greedy
        r = random.random()
        action = random.choice([0, 1, 2] if r < epsilon else maxIndex)
        epsilon -= 0.05

        newXP = xPad + (action - 1 if P - 1 > xPad > 0 else 0)
        newXB, newYB, newDB = updateBall(xBall, yBall, dirBall, newXP)
        reward = rewards(newXB, newYB, newXP)

        Q[xPad][xBall][yBall][dirBall][action] += alpha * (
                reward + max(Q[newXP][newXB][newYB][newDB]) - Q[xPad][xBall][yBall][dirBall][action])

        xPad = newXP
        xBall, yBall, dirBall = newXB, newYB, newDB
        endStep -= 1
    #   print("position pad :", xPad, "position ball :", xBall, yBall, dirBall)

print(Q)




[[[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

   [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

   [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

   [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

   [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
    [-5.00000000e+00 -5.00000000e+00 -5.00000000e+00]
    [ 0.00000000e+00

In [22]:
import gym
import numpy as np
import pyglet
from gym.utils.play import play
import random
class AIQLearning :
    def __init__(self, P, XB, YB, N, epsilon, endStep, alpha, hDivide, wDivide):
        self.XB = XB
        self.YB = YB
        self.P = P
        self.A = 3
        self.D = 4  #  considering : 0 top-left  1 top right ...
        self.Q = np.zeros((self.P, self.XB, self.YB, self.D, self.A), dtype=float)
        self.alpha = alpha
        self.N = N
        self.epsilon = epsilon
        self.endStep = endStep
        # x,y,direction
        self.moveBall = {
            0: (1, 1),  # 45 top right
            1: (1, -1),  # -45 bottom right
            2: (-1, -1),  # -135 bottom left
            3: (-1, 1)  # 135 top-left
        }
        self.hDivide = hDivide
        self.wDivide = wDivide



    def saveQState(self,file):
        print(self.Q)
        with open(file, 'w') as f:
            for p in self.Q:
                for xb in p:
                    for yb in xb:
                        print(yb)
                        np.savetxt(f, delimiter=' ', X=yb, fmt='%d')

    def readQState(self,file):
        self.Q = np.zeros((self.P, self.XB, self.YB, self.D, self.A), dtype=float)
        with open(file, 'r') as f:
            for p in range(self.P):
                for xb in range(self.XB):
                    for yb in range(self.YB):
                        for direction in range(self.D):
                            line = list(int(i) for i in f.readline().split())
                            self.Q[p][xb][yb][direction] = line

    def randomState(self):
        xPad = random.choice(range(self.P))
        xBall = random.choice(range(self.XB-1))
        yBall = random.choice(range(1,self.YB-1))
        dirBall = random.choice(range(4))
        return xPad, xBall, yBall, dirBall

    def learn(self):
        for i in range(self.N):
            epsilon = 1
            endStep = self.endStep
            xPad, xBall, yBall, dirBall = self.randomState()
            while not ((yBall == 0 and dirBall in [1, 2]) or endStep == 0):
                # find the index of the max of the line of Q_state
                maxIndex = np.where(self.Q[xPad][xBall][yBall][dirBall]
                                    == max(self.Q[xPad][xBall][yBall][dirBall]))[0]

                r = random.random()
                action = random.choice([0, 1, 2] if r < epsilon else maxIndex)
                epsilon -= 0.05

                newXP = xPad + (action - 1 if self.P - 1 > xPad > 0 else 0)
                newXB, newYB, newDB = self.updateBall(dirBall, xPad, xBall, yBall)
                reward = self.rewards(xBall, yBall, newXB, newYB, newDB, xPad)

                self.Q[xPad][xBall][yBall][dirBall][action] += self.alpha * (
                        reward + max(self.Q[newXP][newXB][newYB][newDB])
                        - self.Q[xPad][xBall][yBall][dirBall][action])

                xPad = newXP
                xBall, yBall, dirBall = newXB, newYB, newDB
                endStep -= 1
                #print("position pad :", xPad, "position ball :", xBall, yBall, dirBall)


    def rewards(self, xb, yb, newXB, newYB, newDB, xPad):
        if xPad == xb and yb == 0:
            return 3
        return -5 if newYB == 0 and newDB in [1, 2] else 0

    def updateBall(self, d, pad, xB, yB):
        xD, yD = self.moveBall[d]
        nd = d
        nX, nY = xB + xD, yB + yD
        if pad == nX and nY == 0 or self.YB - 1 == nY:
            nd = (d + 2) % 4

        elif (self.XB - 1 == nX and d == 0) or (nX == 0 and d == 3):
            nd = d + (3 if self.XB - 1 == nX and d == 0 else -3)


        elif (self.XB - 1 == nX and d == 1) or (nX == 0 and d == 2):
            nd = d + (1 if self.XB - 1 == nX and d == 1 else -1)
        return nX, nY, nd

    def findNextMove(self,newBall, newPad):
        # find current state
        pad = newPad.discretPad(self.wDivide)
        print("pad" ,newPad.x)
        print("pad discretiwe",pad)
        x, y, dirT = newBall.discretBall(self.hDivide, self.wDivide)
        if (dirT[0] == 0 or dirT[1] ==0):
            return 0
        print("x et y",x,y)
        print(newBall.x, newBall.y)
        dir = list(self.moveBall.keys())[list(self.moveBall.values()).index(dirT)]
        print("dir " ,dir)
        indexes = np.where(self.Q[pad][x][y][dir]
                                    == max(self.Q[pad][x][y][dir]))[0]
        action = random.choice(indexes)
        # because not the same
        if (action == 0):
            return 2
        elif (action == 1):
            return 0
        return 3


class Pad:
    def __init__(self, x):
        self.x = x

    def discretPad(self, width):
        return (self.x // width)


class Ball:
    def __init__(self, coord):
        self.x, self.y = coord
        self.dir = (0, 0)

    def setDirection(self, ball):
        self.dir = (self.x - ball.x, self.y - ball.y)

    # Give xBall,yBall,(direction)
    def discretBall(self, height, width):
        # temporary rescale direction for our usage
        xDir = self.dir[0] if self.dir[0] == 0 else self.dir[0] / abs(self.dir[0])
        yDir = self.dir[1] if self.dir[1] == 0 else self.dir[1] / abs(self.dir[1])
        return (self.x // width),(self.y // height), (xDir, yDir)


class Breakout:
    def __init__(self):
        self.lives = 5
        self.env = gym.make('Breakout-v0')
        self.env.reset()
        self.previousGrid = None
        self.needSpawn = True
        self.nextMove = -1

    def gridReshape(self, grid):
        return grid[74:194:2, 8:152:2, 1]

    # This method is used to set the ball on the grid.
    # Loop until ball spawn
    def spawnBall(self):
        obs, rew, d, inf = self.env.step(1)
        self.previousGrid = self.gridReshape(obs)
        self.env.render()
        ball = self.findBall(self.previousGrid)
        return ball is None

    def lose(self):
        self.needSpawn = True
        self.lives -= 1

    def play(self, IAQState):
        # While IA still has balls : play
        while self.lives > 0:
            if self.needSpawn:
                self.needSpawn = self.spawnBall()
                continue
            ball = self.findBall(self.previousGrid)
            obs, rew, d, inf = self.env.step(0 if self.nextMove == -1 else self.nextMove)
            grid = self.gridReshape(obs)
            self.env.render()
            newBall = self.findBall(grid)
            if newBall is None:
                self.lose()
                continue
            newPad = self.findPad(grid[59])
            newBall.setDirection(ball)
            self.nextMove = IAQState.findNextMove(newBall, newPad)
            self.previousGrid = grid
        self.env.close()

    def findBall(self, grid):
        coordinates = [(colIndex, rowIndex) for rowIndex, rowArray in enumerate(grid)
                       for colIndex, colValue in enumerate(rowArray)
                       if colValue == 72 and self.emptyEdge(rowArray, colIndex)]
        return None if len(coordinates) == 0 else Ball(coordinates[0])

    def emptyEdge(self, rowArray, index):
        if index == 0:
            return rowArray[index + 1] != 72
        if index == 71:
            return rowArray[index - 1] != 72
        return rowArray[index + 1] != 72 and rowArray[index - 1] != 72

    def findPad(self, row):
        pad = [index for index, elem in enumerate(row) if
               elem == 72 and all(pixel == 72 for pixel in row[index:index + 8])]
        return Pad(pad[0])

game = Breakout()
AI = AIQLearning(18,18,15, 10000, 1, 4000, 0.7, 4, 4)
#AI.learn()
AI.readQState('read2.txt')
#AI.saveQState('read2.txt')
game.play(AI)


# Bon j'arrete chu ko la tout marche, faudrait code une petite classe generique pour le learning , j'ai creer les methode en haut, pour que ce soit plus facile a tester apres avec diffenrete configuration.


pad 46
pad discretiwe 11
x et y 8 7
32 28
dir  3
pad 40
pad discretiwe 10
x et y 7 7
30 30
dir  3
pad 37
pad discretiwe 9
x et y 7 7
29 31
dir  3
pad 40
pad discretiwe 10
x et y 6 8
27 33
dir  3
pad 40
pad discretiwe 10
x et y 6 8
25 34
dir  3
pad 41
pad discretiwe 10
x et y 6 9
24 36
dir  3
pad 52
pad discretiwe 13
x et y 5 9
22 38
dir  3
pad 63
pad discretiwe 15
x et y 5 10
20 40
dir  3
pad 68
pad discretiwe 17
x et y 4 10
18 41
dir  3
pad 68
pad discretiwe 17
x et y 4 10
17 43
dir  3
pad 68
pad discretiwe 17
x et y 3 11
15 45
dir  3
pad 68
pad discretiwe 17
x et y 3 11
14 46
dir  3
pad 68
pad discretiwe 17
x et y 3 11
12 47
dir  3
pad 68
pad discretiwe 17
x et y 2 12
11 49
dir  3
pad 68
pad discretiwe 17
x et y 2 12
9 51
dir  3
pad 68
pad discretiwe 17
x et y 2 13
8 52
dir  3
pad 68
pad discretiwe 17
x et y 1 13
6 54
dir  3
pad 68
pad discretiwe 17
x et y 1 14
4 56
dir  3
pad 68
pad discretiwe 17
x et y 0 14
2 58
dir  3
pad 68
pad discretiwe 17
x et y 0 14
1 59
dir  3
pad 68
pad dis